In [1]:
import http.client
import config
import pandas as pd
import json
import folium

In [2]:
conn1 = http.client.HTTPSConnection("app.goflightlabs.com")
conn2 = http.client.HTTPSConnection("app.goflightlabs.com")

conn1.request("GET", f"/flights?access_key={config.api_key}&arrIata=ATL")
conn2.request("GET", f"/flights?access_key={config.api_key}&depIata=ATL")

real_time_arrivals = conn1.getresponse().read().decode('utf-8')
real_time_arrivals = json.loads(real_time_arrivals)

real_time_departures = conn2.getresponse().read().decode('utf-8')
real_time_departures = json.loads(real_time_departures)

In [60]:
df_real_time_departures = pd.json_normalize(real_time_departures['data'])
df_real_time_arrials = pd.json_normalize(real_time_arrivals['data'])
df_real_time_flights = pd.concat([df_real_time_departures, df_real_time_arrials])

In [61]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,flight.number,geography.altitude,geography.direction,geography.latitude,geography.longitude,speed.horizontal,speed.isGround,speed.vspeed,system.squawk,system.updated
0,en-route,B738,A4794E,B738,N388DA,DL,DAL,EWR,KEWR,ATL,...,2644,1813.56,11,40.65,-74.61,492.632,0,0,None,1684422116
1,landed,A319,A977D4,A319,N709UW,AA,AAL,ORD,KORD,ATL,...,2893,0.00,202,41.98,-87.90,0.000,1,0,None,1684422072
2,started,A320,A3F133,A320,N353NW,DL,DAL,SAT,KSAT,ATL,...,1408,0.00,2,33.64,-84.44,5.556,1,0,None,1684422073
3,en-route,A321,A44E24,A321,N377DN,DL,DAL,ONT,KONT,ATL,...,306,10355.60,269,33.47,-89.63,796.360,0,0,None,1684422374
4,landed,B712,AD3C27,B712,N952AT,DL,DAL,OMA,KOMA,ATL,...,2816,0.00,253,41.30,-95.90,0.000,1,0,None,1684421193
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,en-route,A20N,A42A91,A20N,N368FR,F9,FFT,ATL,KATL,SAT,...,2330,11285.20,77,30.33,-92.11,838.956,0,0,None,1684421497
133,en-route,A320,A89CCC,A320,N654NK,NK,NKS,ATL,KATL,DTW,...,3901,10668.00,188,35.77,-83.07,796.360,0,0,None,1684421762
134,en-route,C525,A6658D,C525,N511EC,,,ATL,KATL,DXE,...,XXF,1584.96,61,33.64,-84.85,424.108,0,0,None,1684422077
135,en-route,A333,4CAF9F,A333,EI-HEC,D0,DHK,ATL,KATL,KEF,...,300,11582.40,228,48.59,-69.17,916.740,0,0,None,1684421498


In [62]:
df_airports = pd.read_csv('./data/airports.csv')

In [63]:
df_airports_arrival = df_airports.rename(columns={
    'airportId': 'arrival.airportId',
    'codeIataAirport': 'arrival.iataCode',
    'codeIataCity': 'arrival.codeIataCity',
    'codeIcaoAirport': 'arrival.codeIcaoAirport',
    'codeIso2Country': 'arrival.codeIso2Country',
    'latitudeAirport': 'arrival.latitudeAirport',
    'longitudeAirport': 'arrival.longitudeAirport',
    'nameAirport': 'arrival.nameAirport',
    'nameCountry': 'arrival.nameCountry',
    'timezone': 'arrival.timezone'
})

df_airports_arrival = df_airports_arrival.drop(columns=['GMT', 'geonameId','phone'])


In [64]:
df_real_time_flights = pd.merge(df_real_time_flights, df_airports_arrival, how='left', on='arrival.iataCode')

In [65]:
df_airports_departure = df_airports.rename(columns={
    'airportId': 'departure.airportId',
    'codeIataAirport': 'departure.iataCode',
    'codeIataCity': 'departure.codeIataCity',
    'codeIcaoAirport': 'departure.codeIcaoAirport',
    'codeIso2Country': 'departure.codeIso2Country',
    'latitudeAirport': 'departure.latitudeAirport',
    'longitudeAirport': 'departure.longitudeAirport',
    'nameAirport': 'departure.nameAirport',
    'nameCountry': 'departure.nameCountry',
    'timezone': 'departure.timezone'
})
df_airports_departure = df_airports_departure.drop(columns=['GMT', 'geonameId','phone'])


In [66]:
df_airports_departure

,departure.airportId,departure.iataCode,departure.codeIataCity,departure.codeIcaoAirport,departure.codeIso2Country,departure.latitudeAirport,departure.longitudeAirport,departure.nameAirport,departure.nameCountry,departure.timezone
0,1,AAA,AAA,NTGA,PF,-17.050000,-145.416670,Anaa,French Polynesia,Pacific/Tahiti
1,2,AAB,AAB,YARY,AU,-26.700000,141.041670,Arrabury,Australia,Australia/Brisbane
2,3,AAC,AAC,HEAR,EG,31.133333,33.750000,El Arish International Airport,Egypt,Africa/Cairo
3,4,AAE,AAE,DABB,DZ,36.821392,7.811857,Les Salines,NaN,Africa/Algiers
4,5,AAF,AAF,KAAF,US,29.733334,-84.983330,Apalachicola Regional,United States,America/New_York
...,...,...,...,...,...,...,...,...,...,...
10279,10566,VKX,NaN,KVKX,US,38.748642,-76.955922,Potomac Airfield,United States,NaN
10280,10567,WRN,NaN,YWDG,AU,-30.031667,119.390000,Windarling Airport,Australia,NaN
10281,10568,WXF,NaN,EGVT,GB,51.971901,0.509444,Wethersfield Airport,United Kingdom,NaN
10282,10569,XCD,NaN,LFLH,FR,46.826111,4.817500,Chalon – Champforgeuil Airfield,France,NaN


In [67]:
df_real_time_flights = pd.merge(df_real_time_flights, df_airports_departure, how='left', on='departure.iataCode')

In [68]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,arrival.timezone,departure.airportId,departure.codeIataCity,departure.codeIcaoAirport,departure.codeIso2Country,departure.latitudeAirport,departure.longitudeAirport,departure.nameAirport,departure.nameCountry,departure.timezone
0,en-route,B738,A4794E,B738,N388DA,DL,DAL,EWR,KEWR,ATL,...,America/New_York,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
1,landed,A319,A977D4,A319,N709UW,AA,AAL,ORD,KORD,ATL,...,America/Chicago,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
2,started,A320,A3F133,A320,N353NW,DL,DAL,SAT,KSAT,ATL,...,America/Chicago,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
3,en-route,A321,A44E24,A321,N377DN,DL,DAL,ONT,KONT,ATL,...,America/Los_Angeles,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
4,landed,B712,AD3C27,B712,N952AT,DL,DAL,OMA,KOMA,ATL,...,America/Chicago,411,ATL,KATL,US,33.640068,-84.444030,Hartsfield-jackson Atlanta International,United States,America/New_York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,en-route,A20N,A42A91,A20N,N368FR,F9,FFT,ATL,KATL,SAT,...,America/New_York,6925,SAT,KSAT,US,29.524937,-98.472640,San Antonio International,United States,America/Chicago
314,en-route,A320,A89CCC,A320,N654NK,NK,NKS,ATL,KATL,DTW,...,America/New_York,1936,DTT,KDTW,US,42.207810,-83.356050,Detroit Metropolitan Wayne County,United States,America/Detroit
315,en-route,C525,A6658D,C525,N511EC,,,ATL,KATL,DXE,...,America/New_York,10370,NaN,KMBO,US,NaN,NaN,Bruce Campbell Field,United States\r\n,NaN
316,en-route,A333,4CAF9F,A333,EI-HEC,D0,DHK,ATL,KATL,KEF,...,America/New_York,3765,REK,BIKF,IS,63.997765,-22.624283,Keflavik International,Iceland,Atlantic/Reykjavik


In [69]:
df_airlines = pd.read_csv('./data/airlines.csv')
df_airlines = df_airlines.rename(columns={
    'codeIataAirline': 'airline.iataCode',
    'codeIcaoAirline': 'airline.icaoCode'})
df_airlines = df_airlines.dropna(subset=['airline.iataCode', 'airline.icaoCode'])
df_airlines

,ageFleet,airlineId,callsign,codeHub,airline.iataCode,airline.icaoCode,codeIso2Country,founding,iataPrefixAccounting,nameAirline,nameCountry,sizeAirline,statusAirline,type
0,10.9,1,AMERICAN,DFW,AA,AAL,US,1934,1.0,American Airlines,United States,963,active,scheduled
1,17.0,2,DELTA,ATL,DL,DAL,US,1928,6.0,Delta Air Lines,United States,823,active,"scheduled,division"
2,13.8,3,UNITED,ORD,UA,UAL,US,1931,16.0,United Airlines,United States,715,active,"scheduled,division"
3,12.3,4,SOUTHWEST,DAL,WN,SWA,UM,1967,526.0,Southwest Airlines Co.,United States Minor Outlying Islands,712,active,scheduled
4,6.9,5,CHINA SOUTHERN,CAN,CZ,CSN,CN,2004,784.0,China Southern Airlines,China,498,active,scheduled
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3955,0.0,3957,NaN,YQB,ZX,AAQ,CA,0,NaN,Air Alliance (Canada),Canada,0,merged,scheduled
3956,0.0,3958,AIRCOACH,NaN,ZX,ABL,CA,0,742.0,Air B.C.,Canada,0,historical,NaN
3957,0.0,3959,FLYZOOM,GLA,ZX,UKZ,UK,0,NaN,Zoom Airlines,United Kingdom,0,historical,charter
3958,0.0,3960,ADA AIR,TIA,ZY,ADE,AL,0,121.0,Ada Air,Albania,0,historical,scheduled


In [70]:
df_real_time_flights = pd.merge(df_real_time_flights, df_airlines, how='left',
                                left_on=['airline.iataCode', 'airline.icaoCode'],
                                right_on=['airline.iataCode', 'airline.icaoCode'])


In [71]:
df_real_time_flights

,status,aircraft.iataCode,aircraft.icao24,aircraft.icaoCode,aircraft.regNumber,airline.iataCode,airline.icaoCode,arrival.iataCode,arrival.icaoCode,departure.iataCode,...,callsign,codeHub,codeIso2Country,founding,iataPrefixAccounting,nameAirline,nameCountry,sizeAirline,statusAirline,type
0,en-route,B738,A4794E,B738,N388DA,DL,DAL,EWR,KEWR,ATL,...,DELTA,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division"
1,landed,A319,A977D4,A319,N709UW,AA,AAL,ORD,KORD,ATL,...,AMERICAN,DFW,US,1934.0,1.0,American Airlines,United States,963.0,active,scheduled
2,started,A320,A3F133,A320,N353NW,DL,DAL,SAT,KSAT,ATL,...,DELTA,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division"
3,en-route,A321,A44E24,A321,N377DN,DL,DAL,ONT,KONT,ATL,...,DELTA,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division"
4,landed,B712,AD3C27,B712,N952AT,DL,DAL,OMA,KOMA,ATL,...,DELTA,ATL,US,1928.0,6.0,Delta Air Lines,United States,823.0,active,"scheduled,division"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,en-route,A20N,A42A91,A20N,N368FR,F9,FFT,ATL,KATL,SAT,...,FRONTIER FLIGHT,DEN,UM,1994.0,422.0,"Frontier Airlines, Inc.",United States Minor Outlying Islands,58.0,active,scheduled
314,en-route,A320,A89CCC,A320,N654NK,NK,NKS,ATL,KATL,DTW,...,SPIRIT WINGS,FLL,UM,1980.0,487.0,Spirit Airlines,United States Minor Outlying Islands,84.0,active,scheduled
315,en-route,C525,A6658D,C525,N511EC,,,ATL,KATL,DXE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
316,en-route,A333,4CAF9F,A333,EI-HEC,D0,DHK,ATL,KATL,KEF,...,WORLD EXPRESS,EMA,GB,0.0,936.0,DHL Air,United Kingdom,29.0,active,"scheduled,cargo"
